<a href="https://colab.research.google.com/github/heshumi/NNTI-WS2021-NLP-Project/blob/main/Copy_of_NNTI_final_project_task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1: Word Embeddings (10 points)

This notebook will guide you through all steps necessary to train a word2vec model (Detailed description in the PDF).

## Imports

This code block is reserved for your imports. 

You are free to use the following packages: 

(List of packages)

In [ ]:
# Imports
import pandas as pd
import numpy as np
from math import sqrt
import random
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# 1.1 Get the data (0.5 points)

The Hindi portion HASOC corpus from [github.io](https://hasocfire.github.io/hasoc/2019/dataset.html) is already available in the repo, at data/hindi_hatespeech.tsv . Load it into a data structure of your choice. Then, split off a small part of the corpus as a development set (~100 data points).

If you are using Colab the first two lines will let you upload folders or files from your local file system.

In [ ]:
#TODO: implement!

data = pd.read_csv('hindi_dataset.tsv', sep='\t')
dev = data.iloc[:100,]
dev.head()

,text_id,text,task_1,task_2,task_3
0,hasoc_hi_5556,"बांग्लादेश की शानदार वापसी, भारत को 314 रन पर ...",NOT,NONE,NONE
1,hasoc_hi_5648,सब रंडी नाच देखने मे व्यस्त जैसे ही कोई #शांती...,HOF,PRFN,UNT
2,hasoc_hi_164,तुम जैसे हरामियों के लिए बस जूतों की कमी है शु...,HOF,PRFN,TIN
3,hasoc_hi_3530,"बीजेपी MLA आकाश विजयवर्गीय जेल से रिहा, जमानत ...",NOT,NONE,NONE
4,hasoc_hi_5206,चमकी बुखार: विधानसभा परिसर में आरजेडी का प्रदर...,NOT,NONE,NONE


## 1.2 Data preparation (0.5 + 0.5 points)

* Prepare the data by removing everything that does not contain information. 
User names (starting with '@') and punctuation symbols clearly do not convey information, but we also want to get rid of so-called [stopwords](https://en.wikipedia.org/wiki/Stop_word), i. e. words that have little to no semantic content (and, but, yes, the...). Hindi stopwords can be found [here](https://github.com/stopwords-iso/stopwords-hi/blob/master/stopwords-hi.txt) Then, standardize the spelling by lowercasing all words.
Do this for the development section of the corpus for now.

* What about hashtags (starting with '#') and emojis? Should they be removed too? Justify your answer in the report, and explain how you accounted for this in your implementation.

In [ ]:
#TODO: implement!

with open('stopwords-hi.txt', 'r') as f:
  stopwords = f.read().split()

punct = ':;?!-—-\"\'|।()[]{},./\\“'

def preprocess(x):
  for ch in punct:
    x = x.replace(ch, ' ')

  words = x.split(' ')

  preprocessed=[]

  for word in words: 
    if 1 < len(word) < 30 and word[0]!='@' and word not in stopwords and not word.startswith('http'):
      word.lower().strip()
      preprocessed.append(word)

  return preprocessed

We chose to leave the emojis and hashtags in the text because they might indicate the user's mood which we will be trying to predict. 

In [ ]:
dev['prep_text'] = dev['text'].apply(lambda x: preprocess(x))
dev['prep_text'][:5]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0    [बांग्लादेश, शानदार, वापसी, भारत, 314, रन, रोक...
1    [सब, रंडी, नाच, देखने, व्यस्त, #शांतीदूत, होगा...
2    [तुम, हरामियों, बस, जूतों, कमी, शुक्र, तुम्हार...
3    [बीजेपी, MLA, आकाश, विजयवर्गीय, जेल, रिहा, जमा...
4    [चमकी, बुखार, विधानसभा, परिसर, आरजेडी, प्रदर्श...
Name: prep_text, dtype: object

## 1.3 Build the vocabulary (0.5 + 0.5 points)

The input to the first layer of word2vec is an one-hot encoding of the current word. The output of the model is then compared to a numeric class label of the words within the size of the skip-gram window. Now

* Compile a list of all words in the development section of your corpus and save it in a variable ```V```.

In [ ]:
#TODO: implement!
V = {}

for s in dev['prep_text']:
  for w in s:
    if w in V:
      V[w]+=1
    else: 
      V[w]=1
      
summ = sum(V.values())

* Then, write a function ```word_to_one_hot``` that returns a one-hot encoding of an arbitrary word in the vocabulary. The size of the one-hot encoding should be ```len(v)```.

In [ ]:
#TODO: implement!

# Create a dictionary "{word:1-hot} for a faster access"
# onehot_dict={}

# N = len(V)
# for i, word in enumerate(V.keys()):
#   onehot_dict[word] = np.append(np.append(np.zeros(i), [1]), np.zeros(N-i-1))

# def word_to_one_hot(word):
#   return onehot_dict[word]

#   #TODO: implement!

# # Create a dictionary "{word:1-hot} for a faster access"
onehot_dict={}

N = len(V)
for i, word in enumerate(V.keys()):
  onehot_dict[word] = i

def word_to_one_hot(word):
  i = onehot_dict[word]
  return np.append(np.append(np.zeros(i), [1]), np.zeros(N-i-1))

In [ ]:
# onehot_dict['डालें']

## 1.4 Subsampling (0.5 points)

The probability to keep a word in a context is given by:

$P_{keep}(w_i) = \Big(\sqrt{\frac{z(w_i)}{0.001}}+1\Big) \cdot \frac{0.001}{z(w_i)}$

Where $z(w_i)$ is the relative frequency of the word $w_i$ in the corpus. Now,
* Calculate word frequencies
* Define a function ```sampling_prob``` that takes a word (string) as input and returns the probabiliy to **keep** the word in a context.

In [ ]:
#TODO: implement!

def sampling_prob(word):
  z = V[word]/summ

  return (sqrt(z/0.001)+1)*0.001/z

# 1.5 Skip-Grams (1 point)

Now that you have the vocabulary and one-hot encodings at hand, you can start to do the actual work. The skip gram model requires training data of the shape ```(current_word, context)```, with ```context``` being the words before and/or after ```current_word``` within ```window_size```. 

* Have closer look on the original paper. If you feel to understand how skip-gram works, implement a function ```get_target_context``` that takes a sentence as input and [yield](https://docs.python.org/3.9/reference/simple_stmts.html#the-yield-statement)s a ```(current_word, context)```.

* Use your ```sampling_prob``` function to drop words from contexts as you sample them. 

In [ ]:
#TODO: implement!

def get_target_context(sentence, window_size=5):
  new_sent = [w for w in sentence if random.random() < sampling_prob(w)] # Remove too frequent words 
  # new_sent = sentence # DEBUGGUNG

  for i, w in enumerate(new_sent): # For each word in a sentence (or comment)
    context = [y for y in new_sent[i-window_size:i] + new_sent[i+1:i+window_size+1]]
    yield (w, context) # return its context

# 1.6 Hyperparameters (0.5 points)

According to the word2vec paper, what would be a good choice for the following hyperparameters? 

* Embedding dimension
* Window size

Initialize them in a dictionary or as independent variables in the code block below. 

In [ ]:
# Set hyperparameters
window_size = 5
embedding_size = 300
input_size = len(V) # Onehot vector length
batch_size = 50

# More hyperparameters
learning_rate = 0.05
epochs = 100

# Create a dataset and a data loader

In [ ]:
class HindiDataset(Dataset):
  def __init__(self, df):
    self.df = df

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx): # Returns pairs of 'word - context' in onehot
    cont_list=[]
    word_list=[]
    for word, context in get_target_context(self.df[idx], window_size= window_size):
        word_onehot = word_to_one_hot(word).tolist()

        for cont_word in context:
          cont_onehot = word_to_one_hot(cont_word).tolist()

          word_list.append(word_onehot)
          cont_list.append(cont_onehot)
    
    word_list = torch.tensor(word_list, dtype=torch.double)
    cont_list = torch.tensor(cont_list, dtype = torch.double)

    st = torch.stack([word_list, cont_list])
    st.type(torch.DoubleTensor)
    return st # 2 columns: word_vector tensor, context_vector tensor


In [ ]:
def my_collate(batch):
  data = []
  labels = []
  for b in batch:
    data.append(b[0])
    labels.append(b[1])

  d = torch.cat(data, dim=0)
  l = torch.cat(labels, dim=0)
  
  return (d, l)


In [ ]:
dataset = HindiDataset(dev['prep_text'])
dataloader = DataLoader(dataset, collate_fn = my_collate, batch_size=batch_size, shuffle = False)

# 1.7 Pytorch Module (0.5 + 0.5 + 0.5 points)

Pytorch provides a wrapper for your fancy and super-complex models: [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). The code block below contains a skeleton for such a wrapper. Now,

* Initialize the two weight matrices of word2vec as fields of the class.

* Override the ```forward``` method of this class. It should take a one-hot encoding as input, perform the matrix multiplications, and finally apply a log softmax on the output layer.

* Initialize the model and save its weights in a variable. The Pytorch documentation will tell you how to do that.

In [ ]:
# Create model 

class Word2Vec(nn.Module):
  def __init__(self):
    super().__init__()

    # Hidden layer
    self.fc1 = nn.Linear(input_size, embedding_size)

    # Output layer
    self.fc2 = nn.Linear(embedding_size, input_size)

  def forward(self, one_hot):
    x = self.fc1(one_hot)
    x = F.relu(x)
    x = self.fc2(x)
    # x = F.softmax(x, dim=0) # Softmax is not needed in cosine similarity (in theory)

    return x

In [ ]:
torch.manual_seed(21) # For reproducibility

model = Word2Vec()
model.double()
model.cuda()

# nn.init.xavier_uniform(model.fc1.weight)
# nn.init.xavier_uniform(model.fc2.weight)

nn.init.uniform(model.fc1.weight)
nn.init.uniform(model.fc2.weight)

w1 = model.fc1.weight
w2 = model.fc2.weight

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  # This is added back by InteractiveShellApp.init_path()


# 1.8 Loss function and optimizer (0.5 points)

Initialize variables with [optimizer](https://pytorch.org/docs/stable/optim.html#module-torch.optim) and loss function. You can take what is used in the word2vec paper, but you can use alternative optimizers/loss functions if you explain your choice in the report.

In [ ]:
# Define optimizer and loss
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

criterion = nn.CosineEmbeddingLoss()

# 1.9 Training the model (3 points)

As everything is prepared, implement a training loop that performs several passes of the data set through the model. You are free to do this as you please, but your code should:

* Load the weights saved in 1.6 at the start of every execution of the code block
* Print the accumulated loss at least after every epoch (the accumulate loss should be reset after every epoch)
* Define a criterion for the training procedure to terminate if a certain loss value is reached. You can find the threshold by observing the loss for the development set.

You can play around with the number of epochs and the learning rate.

In [ ]:
# Define train procedure

# load initial weights

def train():
  
  print("Training started")
  
  for epoch in range(epochs):
    acc_loss=0
    words_num=0
    for i, batch in enumerate(dataloader):
      word_t = batch[0].type(torch.DoubleTensor).cuda()
      cont_t = batch[1].cuda()
      
      # print(word_t.size()[0])
      if word_t.size()[0] == 0:
        continue

      outputs = model(word_t)

      loss = criterion(cont_t, outputs, torch.ones(len(cont_t)).cuda()) # Correctly computes 1 - cos(a,b)
      
      acc_loss += loss
      words_num+=1

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    
    print('Epoch: {}; Accumulated mean loss: {}'.format(epoch+1, acc_loss/words_num))

In [ ]:
model.fc1.weight = w1
model.fc2.weight = w2

train()

print("Training finished")

Training started
Epoch: 1; Accumulated mean loss: 0.9728839692814661
Epoch: 2; Accumulated mean loss: 0.9702726657176066
Epoch: 3; Accumulated mean loss: 0.9683225028928424
Epoch: 4; Accumulated mean loss: 0.9662277799014912
Epoch: 5; Accumulated mean loss: 0.9640535799693504
Epoch: 6; Accumulated mean loss: 0.9615411465157433
Epoch: 7; Accumulated mean loss: 0.9583731003130319
Epoch: 8; Accumulated mean loss: 0.9542580942925581
Epoch: 9; Accumulated mean loss: 0.9487543510691454
Epoch: 10; Accumulated mean loss: 0.9420671061406569
Epoch: 11; Accumulated mean loss: 0.9331414699893261
Epoch: 12; Accumulated mean loss: 0.9215793672984522
Epoch: 13; Accumulated mean loss: 0.9075788939191634
Epoch: 14; Accumulated mean loss: 0.892330308684173
Epoch: 15; Accumulated mean loss: 0.8775740147698283
Epoch: 16; Accumulated mean loss: 0.8609746895146704
Epoch: 17; Accumulated mean loss: 0.8431566225505844
Epoch: 18; Accumulated mean loss: 0.8266657067833834
Epoch: 19; Accumulated mean loss: 0.813

In [ ]:
# Save the weights for 100 posts

w1_100 = model.fc1.weight
w2_100 = model.fc2.weight

**Experiment Results**

1) Window Size: 5; lr: 0.05; embedding_size = 30; subsampling=True

Epoch: 1; Error: 0.915

Epoch: 17; Error: 0.858 

Epoch 50: Error 0.8499

Epoch 70: Error 0.8473

Epoch 90: Error 0.8486

Epoch 100: Error 0.8478

2) Window: 3, lr: 0.05; embedding_size = 300; subsampling = False

Epoch 1: 0.906

Epoch 7: 0.8496

Epoch 25: 0.8129

Epoch 50: 0.8194

3) Window: 5, lr: 0.05, embedding = 300, subs = True, batch_size = 500

# 1.10 Train on the full dataset (0.5 points)

Now, go back to 1.1 and remove the restriction on the number of sentences in your corpus. Then, reexecute code blocks 1.2, 1.3 and 1.6 (or those relevant if you created additional ones). 

* Then, retrain your model on the complete dataset.

* Now, the input weights of the model contain the desired word embeddings! Save them together with the corresponding vocabulary items (Pytorch provides a nice [functionality](https://pytorch.org/tutorials/beginner/saving_loading_models.html) for this).

In [ ]:
model.fc1.weight = w1
model.fc2.weight = w2

train()

print("Training finished")

Training started


RuntimeError: ignored

Data size with 100 first points, Batch_size = 100: 13478

Data size with all points, Batch_size = 100: 13617

As it is expected to be!

In [ ]:
torch.save(model.state_dict(), 'Word2Vec.model')

# All data experiments

1) window_size = 5; embedding_size = 300; lr = 0.05;


Epoch: 1; Accumulated mean loss: 0.9383050315444118

Epoch: 2; Accumulated mean loss: 0.9324209570430585

Epoch: 3; Accumulated mean loss: 0.922902618981997

Epoch: 4; Accumulated mean loss: 0.9193773524045055

Epoch: 5; Accumulated mean loss: 0.9180544629896482

Epoch: 6; Accumulated mean loss: 0.9179885761327385

Epoch: 7; Accumulated mean loss: 0.918402046288665

Epoch: 8; Accumulated mean loss: 0.9187660405256706

Epoch: 9; Accumulated mean loss: 0.9189742238755008

# Debugging notes

In [ ]:
model = Word2Vec().cuda()
model.double()

model.load_state_dict(torch.load('Word2Vec.model'))
model.eval()

Word2Vec(
  (fc1): Linear(in_features=20531, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=20531, bias=True)
)

In [ ]:
# Outputs 10 most similar words to the input word. 
  # Works well for window_size = 1, subsampling= false, using only 2 first sentences
             # for w_s = 3, subsampling = false, 2 1st sentences
             # for w_s = 3, subsampling = true, 100 1st sentences

word = torch.from_numpy(word_to_one_hot('#AkashVijayvargiya')).cuda()
word = word.view(1, len(word))

context = model(word)

max = torch.topk(context, 10) # most suitable context (sorted)

for idx in max[1][0]:
  for w, onehot in onehot_dict.items():
      if np.argmax(onehot) == idx:
          print(w)

abpnews
abplive
समर्थक
इंदौर
खुशी
फायरिंग
हर्ष
in
india
#AkashVijayvargiya


In [ ]:
dev['prep_text'][350]

['तन',
 'शोभा',
 'मन',
 'श्रृंगार',
 'नही',
 'मन',
 'शोभा',
 'कोमलता',
 'बनावट',
 'नही',
 'टीका',
 'बिंदी',
 '#चूड़ी',
 'पायल',
 'हों',
 'बात',
 'नही',
 'सुन्दरता',
 'नयनो',
 'लाज',
 'चुनर',
 'भली',
 'प्रेम',
 'नदिया',
 'ऐसी',
 'जिसका',
 'जल',
 'सूखे',
 'नही',
 'धारा',
 'इतनी',
 'निर्मलता',
 'प्रीतम',
 'कभी',
 'रूठे',
 'नही',
 '#हिंदी_शब्द',
 '#शब्द_निधि',
 '#संगीता']

# Possible shortcomings:

1) Biased gradient estimation. Define a train loader and pass the batches, not just single observations